In [3]:
import cv2
import tensorflow as tf
import numpy as np
import mediapipe as mp
import pandas as pd
import datetime
from collections import defaultdict
from scipy.stats import mode

In [4]:
#Load Model
model = tf.keras.models.load_model("expression_model.h5")
model.load_weights("expression_model.weights.h5")

In [6]:
# Inisialisasi objek face mesh dari mediapipe
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Memuat detektor wajah Haar
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Daftar nama kelas ekspresi wajah
class_names = ['Anger', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

# Fungsi preproses gambar
def preprocess_image(image):
    # Resize gambar sesuai input model
    image = cv2.resize(image, (224, 224))
    # Menambahkan dimensi batch
    image = np.expand_dims(image, axis=0)
    return image

# Inisialisasi struktur untuk menyimpan prediksi per detik
predictions_per_second = defaultdict(list)

# Menginisialisasi kamera
cap = cv2.VideoCapture(0)

while True:
    # Membaca frame dari kamera
    ret, frame = cap.read()
    # Membalik frame secara horizontal
    frame = cv2.flip(frame, 1)
    
    # Mendapatkan timestamp saat ini
    current_time = datetime.datetime.now()
    timestamp_str = current_time.strftime("%Y-%m-%d %H:%M:%S")
    
    # Menampilkan tanggal dan jam real-time pada frame dengan ukuran tulisan yang lebih kecil
    cv2.putText(frame, timestamp_str, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    
    # Mendeteksi wajah menggunakan detektor wajah Haar
    faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Jika ada wajah yang terdeteksi
    if len(faces) > 0:
        # Memilih wajah dengan probabilitas terbesar
        (x, y, w, h) = max(faces, key=lambda b: (b[2] * b[3]))
        
        # Memotong wajah dari frame
        face_roi = frame[y:y+h, x:x+w]
        
        # Konversi face_roi ke RGB untuk face mesh
        rgb_face_roi = cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)
        
        # Deteksi face mesh
        results = face_mesh.process(rgb_face_roi)
        
        # Jika ada face mesh yang terdeteksi
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # Menggambar titik-titik landmark face mesh di pojok kanan atas frame
                for landmark in face_landmarks.landmark:
                    landmark_x = int(landmark.x * 100) + frame.shape[1] - 150
                    landmark_y = int(landmark.y * 100) + 20
                    cv2.circle(frame, (landmark_x, landmark_y), 1, (0, 255, 0), -1)
                
                # Preproses gambar wajah
                processed_face = preprocess_image(face_roi)
                
                # Melakukan prediksi
                predictions = model.predict(processed_face)
                
                # Menampilkan probabilitas setiap kelas dengan bar chart
                bar_width = 100
                bar_height = 15
                bar_x_offset = landmark_x - 110
                for i, prob in enumerate(predictions[0]):
                    # Menampilkan nama kelas
                    cv2.putText(frame, class_names[i], (bar_x_offset, landmark_y + 100 + i*40 + 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
                    
                    # Menghitung panjang bar berdasarkan probabilitas
                    bar_length = int(prob * bar_width)
                    # Menggambar bar chart
                    cv2.rectangle(frame, (bar_x_offset, landmark_y + 100 + i*40 + 25), (bar_x_offset + bar_length, landmark_y + 100 + i*40 + bar_height + 25), (255, 255, 255), -1)
                    # Menampilkan probabilitas sebagai persentase di sebelah kanan bar
                    cv2.putText(frame, f'{prob*100:.1f}%', (bar_x_offset + bar_width + 5, landmark_y + 100 + i*40 + 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
        
        # Menampilkan label ekspresi wajah di atas wajah
        predicted_class_idx = np.argmax(predictions)
        predicted_class = class_names[predicted_class_idx]
        cv2.putText(frame, f'Expression: {predicted_class}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        
        # Menambahkan bounding box di sekitar wajah
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2) 

        # Menyimpan hasil prediksi berdasarkan detik
        current_second = current_time.strftime("%Y-%m-%d %H:%M:%S")
        predictions_per_second[current_second].append(predicted_class)
    else:
        # Jika tidak ada wajah yang terdeteksi dalam satu detik
        current_second = current_time.strftime("%Y-%m-%d %H:%M:%S")
        if current_second not in predictions_per_second:
            predictions_per_second[current_second] = ['No Face Detected']
    
    # Menampilkan frame
    cv2.imshow('Expression Detection with Face Mesh', frame)
    
    # Menghentikan loop jika tombol 'Enter' ditekan
    if cv2.waitKey(1) == 13:
        break

# Melepaskan kamera dan menutup jendela OpenCV
cap.release()
cv2.destroyAllWindows()

# Mengkonstruksi dataframe dari hasil prediksi per detik
rows = []
for second, predictions in predictions_per_second.items():
    # Mengambil mode dari prediksi dalam satu detik
    mode_prediction = mode(predictions)[0][0]
    rows.append({'time_stamp': second, 'predicted_expression': mode_prediction})

df = pd.DataFrame(rows)

# Menyimpan dataframe ke file CSV
df.to_csv('predicted_expressions.csv', index=False)

1/1 [==============================] - 0s 41ms/step


C:\Users\Matthew\AppData\Local\Temp\ipykernel_14376\1357129501.py:117: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode_prediction = mode(predictions)[0][0]
C:\Users\Matthew\AppData\Local\Temp\ipykernel_14376\1357129501.py:117: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  mode_prediction = mode(predictions)[0][0]
